In [1]:
import numpy as np
import netCDF4 as nc
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets.embed import embed_minimal_html
import ipywidgets as widgets
import os
os.environ["PROJ_LIB"] = "C:\\Utilities\\Python\\Anaconda\\Library\\share"; #fixr
from mpl_toolkits.basemap import Basemap
import requests

In [2]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug','Sep', 'Oct', 'Nov', 'Dec']

In [3]:
years = np.arange(1980,2021)
years

array([1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
       1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

Downloading Data for salinity and potential temperature

In [7]:
for year in years:
    url = 'https://downloads.psl.noaa.gov/Datasets/godas/salt.'+str(year)+'.nc'
    r = requests.get(url, allow_redirects = True)
    open('salt.'+str(year)+'.nc', 'wb').write(r.content)

In [ ]:
for year in years:
    url = 'https://downloads.psl.noaa.gov/Datasets/godas/pottmp.'+str(year)+'.nc'
    r = requests.get(url, allow_redirects = True)
    open('pottmp.'+str(year)+'.nc', 'wb').write(r.content)

# visualization

In [4]:
os.chdir('C:/Users/tbui0/GODAS data')

In [19]:
def f(Cross_Section, Variables, Year, Month, Depth, lat, lon, Colors):    
    file = nc.Dataset(Variables +'.'+ str(Year)+'.nc')
    data = file.variables[Variables][:]
    lons = file.variables['lon'][:]
    lats = file.variables['lat'][:]
    depths = file.variables['level'][:]
    if Cross_Section == 'latitudinal':
        fig = plt.figure(figsize = (20,8))
        xx,yy = np.meshgrid(lons, 0-depths[0:Depth-1])
        plt.contourf(xx,yy, data[Month-1, 0:Depth-1, lat, :], cmap = Colors)
        plt.title(months[Month-1]+ ' ' +str(int(Year))+ ' '+Variables + ' at Latitude ' + str(lats[lat]) +'', size = 20)
        plt.colorbar()
    elif Cross_Section == 'longitudinal':
        fig =plt.figure(figsize = (12,8))
        xx,yy = np.meshgrid(lats, 0-depths[0:Depth-1])
        plt.contourf(xx,yy, data[Month-1, 0:Depth-1,:,lon], cmap = Colors)
        plt.title(months[Month-1]+ ' ' +str(int(Year))+ ' '+Variables + ' at Longitude ' + str(lons[lon]) +'', size = 20)
        plt.colorbar()
    else:
        xx,yy = np.meshgrid(lons,lats)
        ig = plt.figure(figsize =(20,8))
        m = Basemap(projection = 'cyl', lat_ts = 10, llcrnrlon = 0, urcrnrlon = 360, llcrnrlat = -90, urcrnrlat = 90)
        m.pcolormesh(xx,yy, data[Month-1, Depth-1 ], shading = 'auto',cmap = Colors)
        m.colorbar(location = 'right')
        m.drawcoastlines()
        m.drawmapboundary()
        m.drawparallels(np.arange(-90,90,30), labels = [1,0,0,0])
        m.drawmeridians(np.arange(-180,180,60), labels = [0,0,0,1])
        plt.title(months[Month-1]+ ' ' +str(int(Year))+ ' '+Variables + ' at ' + str(depths[Depth-1]) +'m.', size = 20)
interactive(f, Cross_Section = ['latitudinal', 'longitudinal', 'depth'], Variables = ['pottmp','salt'],
            Year = (1980,2020), Month = (1,12), Depth = (1,40), lat = (0,417), lon = (0,359),
            Colors = ['jet','Greys',  'coolwarm', 'rainbow', 'twilight_shifted'])

interactive(children=(Dropdown(description='Cross_Section', options=('latitudinal', 'longitudinal', 'depth'), …